In [0]:
import torch
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [73]:
DATASET_URL = "https://hub.jovian.ml/wp-content/uploads/2020/05/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

Using downloaded and verified file: ./insurance.csv


In [74]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [0]:
your_name = 'Rohit'

In [0]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [77]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,charges
457,57,female,33.84945,0,no,12314.406052
1050,44,female,41.02005,1,no,8344.060868
56,58,female,35.32575,2,no,14151.663500
311,19,female,27.41700,0,no,1806.871040
1288,20,male,43.73400,2,yes,39878.348640


In [78]:
dataframe.shape

(1271, 6)

In [79]:
num_rows = dataframe.shape[0]
num_rows

1271

In [80]:
num_cols = dataframe.shape[1]
num_cols

6

In [86]:
input_cols = list(dataframe.drop('charges',axis =1))
input_cols

['age', 'sex', 'bmi', 'children', 'smoker']

In [87]:
categorical_cols = list(dataframe.select_dtypes(exclude=['float','int']))
categorical_cols

['sex', 'smoker']

In [99]:
output_cols = [dataframe.columns[-1]]
output_cols

['charges']

In [0]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

In [101]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[57.     ,  0.     , 33.84945,  0.     ,  0.     ],
        [44.     ,  0.     , 41.02005,  1.     ,  0.     ],
        [58.     ,  0.     , 35.32575,  2.     ,  0.     ],
        ...,
        [59.     ,  0.     , 38.628  ,  2.     ,  0.     ],
        [41.     ,  1.     , 31.968  ,  1.     ,  0.     ],
        [36.     ,  0.     , 33.3222 ,  0.     ,  0.     ]]),
 array([[12314.406052 ],
        [ 8344.060868 ],
        [14151.6635   ],
        ...,
        [38387.0323512],
        [ 6533.5244   ],
        [ 5483.062832 ]]))

In [0]:
inputs = torch.tensor(inputs_array, dtype=torch.float32)
targets = torch.tensor(targets_array, dtype=torch.float32)

In [103]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

In [0]:
dataset = TensorDataset(inputs, targets)

In [107]:
val_percent = 0.12 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)

train_size = num_rows - val_size
print(val_size ,train_size)

train_ds, val_ds = random_split(dataset, [train_size, val_size])

152 1119


In [0]:
batch_size = 128

In [0]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

In [110]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[54.0000,  0.0000, 34.1880,  3.0000,  0.0000],
        [45.0000,  0.0000, 42.4963,  0.0000,  0.0000],
        [60.0000,  0.0000, 30.5805,  0.0000,  0.0000],
        [58.0000,  0.0000, 25.2747,  0.0000,  0.0000],
        [33.0000,  0.0000, 36.5190,  2.0000,  0.0000],
        [18.0000,  1.0000, 35.2203,  0.0000,  1.0000],
        [30.0000,  1.0000, 39.1830,  0.0000,  1.0000],
        [51.0000,  1.0000, 28.1940,  0.0000,  0.0000],
        [58.0000,  1.0000, 41.0201,  2.0000,  1.0000],
        [41.0000,  1.0000, 37.2405,  0.0000,  0.0000],
        [19.0000,  1.0000, 36.7410,  0.0000,  0.0000],
        [32.0000,  0.0000, 19.7192,  2.0000,  1.0000],
        [34.0000,  0.0000, 41.4418,  2.0000,  0.0000],
        [57.0000,  0.0000, 28.5714,  2.0000,  0.0000],
        [50.0000,  1.0000, 35.8530,  1.0000,  1.0000],
        [29.0000,  1.0000, 39.4050,  2.0000,  1.0000],
        [39.0000,  1.0000, 50.4273,  2.0000,  0.0000],
        [18.0000,  1.0000, 37.3626,  0.0000,  0.0000],
  

In [113]:
input_size = len(input_cols)
print(input_size)
output_size = len(output_cols)
print(output_size)

5
1


In [0]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size) # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb)                          # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.l1_loss(out, targets)                # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)                           # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

In [0]:
model = InsuranceModel()

In [138]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3727, -0.2155,  0.1890,  0.3767,  0.0774]], requires_grad=True),
 Parameter containing:
 tensor([-0.0557], requires_grad=True)]

In [0]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

In [140]:
result = evaluate(model, val_loader) # Use the the evaluate function
print(result)

{'val_loss': 12234.7265625}


In [141]:
epochs = 10
lr = 1e-3
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [10], val_loss: 11990.6357


In [142]:
epochs = 10
lr = 1e-4
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [10], val_loss: 11966.2207


In [143]:
epochs = 15
lr = 1e-3
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [15], val_loss: 11600.1260


In [144]:
epochs = 20
lr = 1e-4
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 11551.3115


In [145]:
epochs = 20
lr = 5e-7
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 11551.0664


In [0]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [127]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([58.0000,  0.0000, 40.4928,  0.0000,  0.0000])
Target: tensor([12725.2725])
Prediction: tensor([940.0260])


In [128]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([46.0000,  0.0000, 34.1880,  3.0000,  0.0000])
Target: tensor([9791.5166])
Prediction: tensor([764.4180])
